In [1]:
%load_ext rpy2.ipython

In [2]:
import os
homeDir = os.getcwd()+"/"
fileDir = homeDir+"Analysis/"
workDir = "/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/"

In [3]:
%%R -i fileDir -i workDir 
#devtools::install_github(repo="knausb/vcfR")
#install.packages('doParallel')
library(vcfR)
library(doParallel)
#setwd("/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/")
dir.create(fileDir)
setwd(workDir)

# VCF-------------------
vcfDir <- "ensemble/vcf/"
filenames <- dir(vcfDir,"ensemble.snpEff.vcf.gz")
#filenames <- filenames[!grepl(".tbi",filenames)]

vcfRead <- function(filename,vcfDir){
  tumorID <- sub("-ensemble.snpEff.vcf.gz","",filename)
  tmp <- vcfR::read.vcfR(paste0(vcfDir,filename), verbose=FALSE)
  tmpVcf <- tmp@fix
  tmpGt <- tmp@gt
  if(length(colnames(tmpGt))==3){
    if(!grepl("_N\\b",colnames(tmpGt)[3])){ #CHECK THE NAME OF TUMOR AND NORMAL SAMPLES IN VCF!
      tmpGt <- tmpGt[,c(1,3,2)]
    }
  }
  info <- tmpVcf[,"INFO"]
  func <- sapply(info,function(x){
    temp <- unlist(strsplit(x,","))
    paste0(unique(na.omit(unlist(sapply(temp,function(x) unlist(strsplit(x,"|",fixed=TRUE))[2])))),collapse=",")
  })
  func <- unname(func)
  callers <- sapply(info,function(x){
    temp <- unlist(strsplit(x,";"))
    sub("CALLERS=","",temp[grepl("CALLERS=",temp)])
  })
  callers <- unname(callers)
  id <- paste0(tumorID,"-",tmpVcf[,1],":",tmpVcf[,2])
  if(length(colnames(tmpGt))==3){
    tmp <- data.frame(tmpVcf[,c(1,2,4,5,7)],tmpGt,tumorID,info,func,callers,id,stringsAsFactors=FALSE)
  } else {
    tmp <- data.frame(tmpVcf[,c(1,2,4,5,7)],tmpGt,normal=NA,tumorID,info,func,callers,id,stringsAsFactors=FALSE)
  }
  colnames(tmp) <- c("CHROM","POS","REF","ALT","FILTER","FORMAT","Tumor","Normal","Tumor_ID","snpEff","snpEff_func","caller","id")
  rownames(tmp) <- NULL
  return(tmp)
}

vcf <- lapply(filenames,function(x) vcfRead(x,vcfDir))
vcf <- do.call(rbind,vcf)
nrow(vcf) #306237


R[write to console]: 
   *****       ***   vcfR   ***       *****
   This is vcfR 1.12.0 
     browseVignettes('vcfR') # Documentation
     citation('vcfR') # Citation
   *****       *****      *****       *****


R[write to console]: Loading required package: foreach

R[write to console]: Loading required package: iterators

R[write to console]: Loading required package: parallel



[1] 306237


## CORRECT VCF

In [4]:
%%R 
# CORRECT FOR NA ALT
table(is.na(vcf$ALT))
# vcf <- vcf[!is.na(vcf$ALT),]

# CORRECT FOR MULTIPLE ALT
table(grepl(",",vcf$ALT))
tmp <- sapply(vcf$ALT,function(x) unlist(strsplit(x,",",fixed=TRUE))[1])
vcf$ALT <- tmp

# # CORRECT FOR NA LINES
#sum(is.na(vcf))
#vcf <- vcf[!is.na(vcf)]
#sum(is.na(vcf))

## SELECT CODING/SPLICING

In [5]:
%%R 
# SELECT CODING/SPLICING
tmp <- unique(vcf$snpEff_func)
tmp <- unique(unlist(sapply(tmp,function(x) unlist(strsplit(x,",")))))
tmp <- unique(unlist(sapply(tmp,function(x) unlist(strsplit(x,"&")))))
tmp <- tmp[!grepl("ENS",tmp)]
tmp <- tmp[!grepl("=",tmp)]
sort(tmp)
sel <- c("5_prime_UTR_premature_start_codon_gain_variant","5_prime_UTR_truncation", #NEW
         "3_prime_UTR_variant", "5_prime_UTR_variant", # Added
         "bidirectional_gene_fusion","conservative_inframe_deletion","conservative_inframe_insertion",
         "disruptive_inframe_deletion","disruptive_inframe_insertion",
         "exon_loss_variant","initiator_codon_variant","gene_fusion",
         "frameshift_variant","missense_variant","splice_acceptor_variant",
         "splice_donor_variant","splice_region_variant","start_lost","stop_gained",
         "stop_lost","stop_retained_variant")
setdiff(tmp,sel)
setdiff(sel,tmp)
sel <- sapply(sel,function(x) grep(x,vcf$snpEff_func))
sel <- unique(unlist(sel))
sel <- sort(sel)
sum(is.na(sel))
nrow(vcf) #227168
rownames(vcf) <- NULL
length(sel) #36938
vcf <- vcf[sel,]
sort(table(vcf$Tumor_ID))

callerN <- sapply(vcf$caller,function(x) length(unlist(strsplit(x,",",fixed=TRUE))))
callerN <- unname(callerN)
vcf <- data.frame(vcf,callerN,stringsAsFactors = FALSE)
table(vcf$callerN)
#   1     2     3     4     5     6 
# 29690  3753   704   328   438  2025  
rownames(vcf) <- NULL
#----------------------------------------------

save(vcf,file = paste(fileDir,"vcf.rda",sep=""))

## COUNTS and ALLELE FREQUENCY

In [6]:
%%R 
# COUNTS and ALLELE FREQUENCY -----------------

countsVcf <- function(vcf,i){
  Tmp <- rep(NA,4)
  names(Tmp) <- c("t_ref_count","t_alt_count","n_ref_count","n_alt_count")
  vcfLine <- as.character(vcf[i,])
  names(vcfLine) <- colnames(vcf)
  caller <- unlist(strsplit(vcfLine["caller"],",",fixed=TRUE))[1]
  if(caller=="varscan"){
    tmp <- unlist(strsplit(vcfLine["Tumor"],":"))
    names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))
    Tmp["t_alt_count"] <- as.numeric(tmp["AD"])
    Tmp["t_ref_count"] <- as.numeric(tmp["RD"])
    tmp <- unlist(strsplit(vcfLine["Normal"],":"))
    names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))
    Tmp["n_alt_count"] <- as.numeric(tmp["AD"])
    Tmp["n_ref_count"] <- as.numeric(tmp["RD"])
  } else {
    tmp <- unlist(strsplit(vcfLine["Tumor"],":"))
    names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))
    if(tmp["AD"]!=".") Tmp[c("t_ref_count","t_alt_count")] <- as.numeric(unlist(strsplit(tmp["AD"],","))[1:2])
    if(!is.na(vcfLine["Normal"])) {
      tmp <- unlist(strsplit(vcfLine["Normal"],":"))
      names(tmp) <- unlist(strsplit(vcfLine["FORMAT"],":"))[1:length(tmp)]
      if(tmp["AD"]!=".") Tmp[c("n_ref_count","n_alt_count")] <- as.numeric(unlist(strsplit(tmp["AD"],","))[1:2])
    }
  }
  return(Tmp)
}

cl <- makePSOCKcluster(60,outfile="")
registerDoParallel(cl)
tmp <- foreach(i = 1:nrow(vcf)) %dopar% {
  tmp <- countsVcf(vcf,i)
  return(tmp)
}
stopCluster(cl)

tmp <- do.call(rbind,tmp)
dim(tmp) #36938     4

counts <- cbind(tmp[,1:2],NA,NA,tmp[,3:4],NA,NA)
colnames(counts) <- c("t_ref_count","t_alt_count","t_depth","t_vaf","n_ref_count","n_alt_count","n_depth","n_vaf")
counts[,"t_depth"] <- counts[,"t_alt_count"] + counts[,"t_ref_count"]
counts[,"t_vaf"] <- round(counts[,"t_alt_count"]/counts[,"t_depth"],2)
counts[,"n_depth"] <- counts[,"n_alt_count"] + counts[,"n_ref_count"]
counts[,"n_vaf"] <- round(counts[,"n_alt_count"]/counts[,"n_depth"],2)
rownames(counts) <- NULL

#View(counts[sample(1:nrow(counts),100),])

vcf <- data.frame(vcf, counts, stringsAsFactors = FALSE)

table(cond <- is.na(vcf$n_depth)&!is.na(vcf$Normal))
# FALSE  TRUE 
# 27894  9044 
# table(cond <- is.na(vcf$n_depth))

tmp <- vcf[cond,]

# table(tmp$caller)
# freebayes 
# 9044

vcf <- vcf[!cond,]

rownames(vcf) <- NULL
nrow(vcf) #27894
rm(counts)

#----------------------------------------------
# save(vcf,file = "Analysis/vcf.rda")

## FP FILTER

In [41]:
fpfilterDir = fileDir + "fpfilter/"

In [9]:
%%R -i fpfilterDir
# FP FILTER
#system("mkdir Analysis/fpfilter")
#setwd("Analysis/fpfilter/")
dir.create(fpfilterDir)
setwd(fpfilterDir)

vcf <- data.frame(vcf,fpFid="",stringsAsFactors=FALSE)
patients <- unique(vcf$Tumor_ID)
for(i in 1:length(patients)){
  print(patients[i])
  name <- patients[i]
  var <- vcf[vcf$Tumor_ID==name,1:4]
  var[nchar(var$ALT)>1,"ALT"] <- paste0("+",substring(var[nchar(var$ALT)>1,"ALT"],2))
  var[nchar(var$REF)>1,"POS"] <- as.numeric(var[nchar(var$REF)>1,"POS"])+1
  var[nchar(var$REF)>1,"ALT"] <- paste0("-",substring(var[nchar(var$REF)>1,"REF"],2))
  var[nchar(var$REF)>1,"REF"] <- substring(var[nchar(var$REF)>1,"REF"],2,2)
  vcf[vcf$Tumor_ID==name,"fpFid"] <- paste0(name,"-",var[,1],":",var[,2],"-",var[,4])
  write.table(var,file=paste0(name,".var"),sep="\t",quote=FALSE,row.names=FALSE,col.names=FALSE)
  # n <- ceiling(nrow(var)/60)
  # system(paste0("split -l ",n," --additional-suffix .var ",paste0(name,".var")))
  loc <- paste(var$CHROM,var$POS,var$POS)
  write.table(loc,file=paste0(name,".loc"),quote=FALSE,row.names=FALSE,col.names=FALSE)
  # system(paste0("split -l ",n," --additional-suffix .loc ",paste0(name,".loc")))
}


[1] "A1783"
[1] "A9155"
[1] "A9878"
[1] "AT2550"
[1] "AT2822"
[1] "AT4415"
[1] "AT4808"
[1] "C0288"
[1] "C0334"
[1] "C1572"
[1] "C8448"
[1] "s1_3076_D"
[1] "s1_3295_D"
[1] "s1_8588_D"
[1] "s2_6088_D"
[1] "s2_6339_D"
[1] "s2_8811_D"
[1] "s3_5832_D"
[1] "s3_7299_D"
[1] "s3_7979_D"


In [61]:
os.chdir(fpfilterDir)

In [62]:
%%bash
fasta=/storage/gluster/vol1/bcbio/genomes/Hsapiens/hg19/bwa/hg19.fa
bamreadcount=/storage/gluster/vol1/SHARED/NGSTOOLS/bam-readcounts/bam-readcount/build/bin/bam-readcount

#cd {fpfilterDir}

for file in *.loc
do
	echo $file
	name=${file%.loc}
	echo $name
	echo "$name bam-readcount"
	bam=`ls /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/*/*/${name}_T_recal.bam`
	echo $bam
    echo '\n'
	$bamreadcount -f $fasta -w 1 -l $file $bam  > ${name}.readcount &
done  

A1783.loc
A1783
A1783 bam-readcount
/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/ABT414_Flank/ABT414_Flank/A1783_T_recal.bam /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/CNV/tumorbam/A1783_T_recal.bam
\n
A9155.loc
A9155
A9155 bam-readcount
/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/ABT414_Flank/ABT414_Flank/A9155_T_recal.bam /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/CNV/tumorbam/A9155_T_recal.bam
\n
A9878.loc
A9878
A9878 bam-readcount
/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/ABT414_Flank/ABT414_Flank/A9878_T_recal.bam /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/CNV/tumorbam/A9878_T_recal.bam
\n
AT2550.loc
AT2550
AT2550 bam-readcount
/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/ABT414_Flank/ABT414_Flank/AT2550_T_recal.bam /storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/CNV/tumorbam/AT2550_T_recal.bam
\n
AT2822.loc
AT2822
AT2822 bam-readcount
/sto

Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
Minimum mapping quality is set to 0
The previous warning has been emitted 1 times and will be disabled.
The previous warning has been emitted 1 times and will be disabled.
The previous warning has been emitted 1 times and will be disabled.
The previous warning has been emitted 1 times and will be disabled.
The prev

In [58]:
%%bash
fpfilter=/storage/gluster/vol1/data/PUBLIC/Tools/VARSCAN/fpfilter.pl

for file in *.var
do
	echo $file
	perl $fpfilter ${file} ${file%.var}.readcount --output-basename ${file%.var}.fpfilter > ${file%.var}.log &
done  

A1783.var
A9155.var
A9878.var
AT2550.var
AT2822.var
AT4415.var
AT4808.var
C0288.var
C0334.var
C1572.var
C8448.var
s1_3076_D.var
s1_3295_D.var
s1_8588_D.var
s2_6088_D.var
s2_6339_D.var
s2_8811_D.var
s3_5832_D.var
s3_7299_D.var
s3_7979_D.var


In [63]:
%%R

setwd(fpfilterDir)
failfiles <- dir(path=".",pattern = "fpfilter.fail")
FPfail <- NULL
for(i in 1:length(failfiles)){
  print(i)
  patient <- unlist(strsplit(failfiles[i],".",fixed=TRUE))[1]
  tmp <- readLines(failfiles[i])
  tmp <- sapply(tmp,function(x){
    Tmp <- unlist(strsplit(x,"\t"))
    length(Tmp) <- 9
    Tmp
  })
  tmp <- t(tmp)
  failID <- paste0(patient,"-",tmp[,1],":",tmp[,2],"-",tmp[,4])
  tmp <- cbind(failID,tmp[,9])
  rownames(tmp) <- failID
  FPfail <- rbind(FPfail,tmp)
}

passfiles <- dir(path=".",pattern = "fpfilter.pass")
FPpass <- NULL
for(i in 1:length(passfiles)){
  print(i)
  patient <- unlist(strsplit(passfiles[i],".",fixed=TRUE))[1]
  tmp <- read.delim(passfiles[i],header=FALSE,as.is=TRUE)
  tmp <- paste0(patient,"-",tmp[,1],":",tmp[,2],"-",tmp[,4])
  FPpass <- c(FPpass,tmp)
}

length(FPpass) #8967
nrow(FPfail) #18886

length(FPpass)+nrow(FPfail) #27853

all <- c(FPpass,FPfail[,1])
table(vcf$fpFid%in%all)
#View(vcf[!vcf$fpFid%in%all,])

vcf <- data.frame(vcf,FPfilter=".",FPfail=".",stringsAsFactors=FALSE)
vcf[vcf$fpFid%in%FPpass,"FPfilter"] <- "PASS"
vcf[vcf$fpFid%in%rownames(FPfail),"FPfilter"] <- "FAIL"
vcf[vcf$FPfilter=="FAIL","FPfail"] <- FPfail[vcf[vcf$FPfilter=="FAIL","fpFid"],2]

vcf[is.na(vcf$FPfail),"FPfail"] <- ""
vcf[vcf$FPfail=="","FPfilter"] <- "no_readcounts" # alt absent in the bam readcount file
vcf[vcf$FPfail=="","FPfail"] <- "."
vcf[vcf$FPfilter==".","FPfilter"] <- "no_readcounts"# alt = 0 in the bam readcount file
table(vcf$FPfilter)
# FAIL no_readcounts          PASS 
# 15577          3350          8967  
rm(FPpass,FPfail,all)

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20


In [64]:
%%R

# # VirtualNormalCorrection----------
# system("mkdir VN")
# setwd("VN/")
# ID <- 1:nrow(vcf)
# VCF <- data.frame(vcf[,1:2],ID,vcf[,3:4],stringsAsFactors = FALSE)
# write.table(VCF,sep="\t",row.names = FALSE,quote=FALSE,file = "VNinput.vcf")
# n <- ceiling(nrow(VCF)/60)
# system(paste0("split -l ",n," --additional-suffix .vcf VNinput.vcf"))
# 
# # UNIX
# # parallelVN.sh
# # parallelVN_combineRes.sh
# 
# VNoutput <- read.delim("VNoutput_main.tsv",as.is=TRUE)
# rownames(vcf) <- VCF$ID
# vcf <- data.frame(vcf,idNum=VCF$ID,stringsAsFactors = FALSE)
# vcf <- data.frame(vcf[VNoutput$xRef,],VNoutput[,9:12],stringsAsFactors = FALSE)
# # threshold 5% (~ 22/433) - threshold_highconf 10% (~ 44/433)
# table(vcf$VN_occurrences<22&vcf$VN_fullycalled_count>=44)
# # FALSE   TRUE 
# # 220620  84644 
# table(vcf$VN_occurrences<3&vcf$VN_fullycalled_count>=110)
# # FALSE   TRUE 
# # 245383  59881 
# table(vcf$VN_occurrences<3) # 0.7% (3/433)
# # FALSE   TRUE 
# # 233414  71850 
# # table(is.na(vcf$VN_occurrences))
# 
# vcf$vn <- vcf$VN_occurrences<3
# table(vcf$vn)
# # FALSE   TRUE 
# # 233414  71850 
# 
# rm(VCF,VNoutput,tmp)

setwd("/storage/gluster/vol1/data/PUBLIC/SCAMBIO/ABT414_WES_Analysis/Analysis/")
save(vcf,file = paste(fileDir,"vcf.rda",sep=""))

# STEP 2 - ANNOTATION AND DOWNSTREAM ANALYSIS #

# ANNOVAR Annotation

In [70]:
os.chdir(fileDir) 

In [72]:
%%R
ann.in <- vcf[,c(1,2,2,3,4)]
colnames(ann.in) <- c("Chr","Start","End","Ref","Alt")

cond <- nchar(ann.in[,"Ref"])>1&nchar(ann.in[,"Alt"])==1
ann.in[cond,"Start"] <- as.numeric(ann.in[cond,"Start"])+1
ann.in[cond,"Ref"] <- substring(ann.in[cond,"Ref"],2,nchar(ann.in[cond,"Ref"]))
ann.in[cond,"Alt"] <- "-"
ann.in[cond,"End"] <- as.numeric(ann.in[cond,"Start"])+nchar(ann.in[cond,"Ref"])-1

cond <- nchar(ann.in[,"Alt"])>1&nchar(ann.in[,"Ref"])==1
ann.in[cond,"Ref"] <- "-"
ann.in[cond,"Alt"] <- substring(ann.in[cond,"Alt"],2,nchar(ann.in[cond,"Alt"]))

cond <- nchar(ann.in[,"Ref"])>1&nchar(ann.in[,"Alt"])>1
ann.in[cond,"End"] <- as.numeric(ann.in[cond,"Start"])+nchar(ann.in[cond,"Ref"])-1

write.table(ann.in,file=paste(fileDir,"ann.in",sep=""),col.names=FALSE,row.names=FALSE,quote=FALSE,sep="\t")

In [74]:
ls

ann.in  ann.out.hg19_multianno.txt  fpfilter/  vcf.rda


In [73]:
%%bash
/storage/gluster/vol1/data/PUBLIC/Tools/annovar2017Jul16/table_annovar.pl ann.in \
/storage/gluster/vol1/data/PUBLIC/Tools/annovar2017Jul16/humandb/ -buildver hg19 -out ann.out \
-remove -protocol refGene,avsnp150,snp138NonFlagged,\
exac03,1000g2015aug_all,esp6500siv2_all,kaviar_20150923,hrcr1,\
cosmic80,clinvar_20170905,\
dbnsfp33a,dbscsnv11,dann,eigen,gerp++gt2,cadd  \
-operation g,f,f,f,f,f,f,f,f,f,f,f,f,f,f,f -nastring . --thread 80

-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver hg19 -dbtype refGene -outfile ann.out.refGene -exonsort ann.in /storage/gluster/vol1/data/PUBLIC/Tools/annovar2017Jul16/humandb/ -thread 80>
NOTICE: Output files were written to ann.out.refGene.variant_function, ann.out.refGene.exonic_variant_function
NOTICE: the queryfile contains 27894 lines
NOTICE: threading is disabled for gene-based annotation on file with less than 1000000 input lines
NOTICE: Reading gene annotation from /storage/gluster/vol1/data/PUBLIC/Tools/annovar2017Jul16/humandb/hg19_refGene.txt ... Done with 63481 transcripts (including 15216 without coding sequence annotation) for 27720 unique genes
NOTICE: Processing next batch with 27894 unique variants in 27894 input lines
NOTICE: Reading FASTA sequences from /storage/gluster/vol1/data/PUBLIC/Tools/annovar2017Jul16/humandb/hg19_

In [77]:
%%R
ann.out <- read.delim("ann.out.hg19_multianno.txt",as.is=TRUE)
paste0(colnames(ann.out),collapse="','")
tmp <- c('Chr','Start','End','Ref','Alt','Func.refGene','Gene.refGene','GeneDetail.refGene','ExonicFunc.refGene','AAChange.refGene', #refGene
         'avsnp150', #avsnp150
         'snp138NonFlagged', #snp138NonFlagged
         'ExAC_ALL','ExAC_AFR','ExAC_AMR','ExAC_EAS','ExAC_FIN','ExAC_NFE','ExAC_OTH','ExAC_SAS', #exac03
         'X1000g2015aug_all', #1000g2015aug_all
         'esp6500siv2_all', #esp6500siv2_all
         'Kaviar_AF','Kaviar_AC','Kaviar_AN', #kaviar_20150923
         'HRC_AF','HRC_AC','HRC_AN','HRC_non1000G_AF','HRC_non1000G_AC','HRC_non1000G_AN', #hrcr1
         'cosmic80', #cosmic80
         'CLINSIG','CLNDBN','CLNACC','CLNDSDB','CLNDSDBID', #clinvar_20170905
         'SIFT_score','SIFT_converted_rankscore','SIFT_pred', #dbnsfp33a
         'Polyphen2_HDIV_score','Polyphen2_HDIV_rankscore','Polyphen2_HDIV_pred', #dbnsfp33a
         'Polyphen2_HVAR_score','Polyphen2_HVAR_rankscore','Polyphen2_HVAR_pred', #dbnsfp33a
         'LRT_score','LRT_converted_rankscore','LRT_pred', #dbnsfp33a
         'MutationTaster_score','MutationTaster_converted_rankscore','MutationTaster_pred', #dbnsfp33a
         'MutationAssessor_score','MutationAssessor_score_rankscore','MutationAssessor_pred', #dbnsfp33a
         'FATHMM_score','FATHMM_converted_rankscore','FATHMM_pred', #dbnsfp33a
         'PROVEAN_score','PROVEAN_converted_rankscore','PROVEAN_pred', #dbnsfp33a
         'VEST3_score','VEST3_rankscore','MetaSVM_score','MetaSVM_rankscore','MetaSVM_pred', #dbnsfp33a
         'MetaLR_score','MetaLR_rankscore','MetaLR_pred', #dbnsfp33a
         'M.CAP_score','M.CAP_rankscore','M.CAP_pred', #dbnsfp33a
         'CADD_raw','CADD_raw_rankscore','CADD_phred', #dbnsfp33a
         'DANN_score','DANN_rankscore', #dbnsfp33a
         'fathmm.MKL_coding_score','fathmm.MKL_coding_rankscore','fathmm.MKL_coding_pred', #dbnsfp33a
         'Eigen_coding_or_noncoding','Eigen.raw','Eigen.PC.raw', #dbnsfp33a
         'GenoCanyon_score','GenoCanyon_score_rankscore', #dbnsfp33a
         'integrated_fitCons_score','integrated_fitCons_score_rankscore','integrated_confidence_value', #dbnsfp33a
         'GERP.._RS','GERP.._RS_rankscore', #dbnsfp33a
         'phyloP100way_vertebrate','phyloP100way_vertebrate_rankscore','phyloP20way_mammalian','phyloP20way_mammalian_rankscore', #dbnsfp33a
         'phastCons100way_vertebrate','phastCons100way_vertebrate_rankscore','phastCons20way_mammalian','phastCons20way_mammalian_rankscore', #dbnsfp33a
         'SiPhy_29way_logOdds','SiPhy_29way_logOdds_rankscore', #dbnsfp33a
         'Interpro_domain', #dbnsfp33a
         'GTEx_V6_gene','GTEx_V6_tissue', #dbnsfp33a
         'dbscSNV_ADA_SCORE','dbscSNV_RF_SCORE', #dbscsnv11 SPLICE SITE
         'dann', #dann
         'Eigen', #eigen
         'gerp..gt2', #gerp++gt2
         'CADD','CADD_Phred') #cadd

sel <- c('Chr','Start','End','Ref','Alt',
         'Func.refGene','Gene.refGene','GeneDetail.refGene','ExonicFunc.refGene','AAChange.refGene', 'Interpro_domain',
         'avsnp150','snp138NonFlagged',
         'ExAC_ALL','X1000g2015aug_all','esp6500siv2_all','Kaviar_AF','HRC_AF',
         'cosmic80','CLINSIG','CLNDBN','CLNACC','CLNDSDB','CLNDSDBID',
         'SIFT_pred','Polyphen2_HDIV_pred','MutationTaster_pred','PROVEAN_pred',
         'dbscSNV_ADA_SCORE','dbscSNV_RF_SCORE',
         'dann','Eigen','gerp..gt2','CADD','CADD_Phred')

ann.out <- ann.out[,sel]
ann.out[ann.out$ExonicFunc.refGene!=".","Func.refGene"] <- ann.out[ann.out$ExonicFunc.refGene!=".","ExonicFunc.refGene"]
ann.out[ann.out$AAChange.refGene!=".","GeneDetail.refGene"] <- ann.out[ann.out$AAChange.refGene!=".","AAChange.refGene"]
ann.out <- ann.out[,!colnames(ann.out)%in%c("ExonicFunc.refGene","AAChange.refGene")]

# Final table (MAF-like)

In [ ]:
%%R
colnames(vcf)
sel <- c("snpEff","snpEff_func","caller","callerN","t_depth","t_ref_count", "t_alt_count",
         "n_depth","n_ref_count", "n_alt_count", "t_vaf","n_vaf","FPfilter","FPfail"#,
         #"VN_occurrences","VN_frequency","VN_fullycalled_count","VN_fullycalled_frequency","vn"
)
setdiff(colnames(vcf),sel)
setdiff(sel,colnames(vcf))

# tumor <- vcf$Tumor_ID
# info <- NULL
# for(i in 1:length(tumor)){
#   print(i)
#   tmp <- samples[samples$TUMOR_WES_ID==tumor[i],c("Patient_ID", "TUMOR_WES_ID", "NORMAL_WES_ID")]
#   info <- rbind(info,tmp)
# }
# colnames(info) <- c("Patient_ID","Tumor_ID","Normal_ID")

final <- data.frame(Patient_ID = vcf[, "Tumor_ID"], Type= "SNV", ann.out, vcf[, sel], stringsAsFactors = FALSE)
final[final$Ref=="-","Type"] <- "INS"
final[final$Alt=="-","Type"] <- "DEL"

id <- paste0(final$Patient_ID,".",final$Chr,".",final$Start,".",final$End,".",final$Alt)
sum(duplicated(id)) #11
final <- data.frame(final,duplicated=duplicated(id),stringsAsFactors = FALSE)
final <- final[!duplicated(id),]
Final <- final
rownames(Final) <- NULL
# save(Final,file="Final.tmp.rda")

# PoNs FILTER (reference_PoNs_final_summed_tokens.hist)----------
source("/storage/gluster/vol1/data/BIOINFO/fulvio/Columbia/ponFilter.R")
Final <- ponFilter(maf=Final, ncores=80, threshold=-2.5, 
                   pondir="/storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/pon_by_chr/")
table(Final$PON_filter_pass)
# FALSE  TRUE 
# 19207  8676 

[1] "reading pon file /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/pon_by_chr//pon_chr1.rds"


R[write to console]: Loading required package: doMC



[1] "reading pon file /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/pon_by_chr//pon_chr2.rds"
[1] "reading pon file /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/pon_by_chr//pon_chr3.rds"
[1] "reading pon file /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/pon_by_chr//pon_chr4.rds"
[1] "reading pon file /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/firecloud_PoNs/pon_by_chr//pon_chr5.rds"


# GENE DESCRIPTION

In [ ]:
%%R
library(biomaRt)
ensembl <- useMart("ensembl",dataset="hsapiens_gene_ensembl")
genes <- unique(Final$Gene.refGene)
# tmp <- listAttributes(ensembl)
# tmp[grep("description",tmp$name),]
# tmp <- listFilters(ensembl)
# tmp[grep("external_gene_name",tmp$name),]
info <- getBM(attributes = c("external_gene_name","description"),filters = c("external_gene_name"),values = genes, mart = ensembl)
Final <- data.frame(Description=".",Final,stringsAsFactors = FALSE)
info <- info[!duplicated(info$external_gene_name),]
rownames(info) <- info$external_gene_name
Final$Description <- info[Final$Gene.refGene,"description"]
table(is.na(Final$Description))


sel <- grep(";",Final$Gene.refGene)
genes <- Final$Gene.refGene[sel]
genes <- sapply(genes,function(x) unlist(strsplit(x,";")))
genes <- unique(unlist(genes))
info <- getBM(attributes = c("external_gene_name","description"),filters = c("external_gene_name"),values = genes, mart = ensembl)
description <- sapply(Final$Gene.refGene[sel],function(x){
  tmp <- unlist(strsplit(x,";"))
  if(tmp[1]%in%info$external_gene_name){
    a <- info[info$external_gene_name==tmp[1],"description"][1]
  } else {
    a <- "."
  }
  if(tmp[2]%in%info$external_gene_name){
    b <- info[info$external_gene_name==tmp[2],"description"][1]
  } else {
    b <- "."
  }
  paste0(a," ; ",b)
})

Final$Description[sel] <- description
table(is.na(Final$Description))

In [ ]:
# # HGVS from SNPEFF----------
# snpEff <- sapply(Final$snpEff,function(x){
#   temp <- unlist(strsplit(x,"ANN"))[2]
#   # temp <- unlist(strsplit(temp,","))[1]
#   temp <- unlist(strsplit(temp,","))
#   if(sum(grepl("p.",temp,fixed=TRUE))>0){
#     temp <- temp[grep("p.",temp,fixed=TRUE)][1]
#   } else temp <- temp[1]
#   info <- unlist(strsplit(temp,"|",fixed=TRUE))
#   HGVSc <- info[grep("c.",info,fixed=TRUE)][1]
#   HGVSp <- info[grep("p.",info,fixed=TRUE)][1]
#   return(c(HGVSc,HGVSp))
# })
# dim(snpEff)
# snpEff <- t(snpEff)
# colnames(snpEff) <- c("HGVSc","HGVSp")
# rownames(snpEff) <- NULL
# Final <- data.frame(snpEff,Final,stringsAsFactors = FALSE)
# 
# #----------------------------------------------

# Annotation by variant classification

In [ ]:
%%R
Final <- data.frame(effectSnpEff=".",effectAnnovar=".",Final,stringsAsFactors = FALSE)

#snpEff
tmp <- unique(Final$snpEff_func)
tmp <- unique(unlist(sapply(tmp,function(x) unlist(strsplit(x,",")))))
tmp <- unique(unlist(sapply(tmp,function(x) unlist(strsplit(x,"&")))))
sort(tmp)
coding <- c("conservative_inframe_deletion","conservative_inframe_insertion",
            "disruptive_inframe_deletion","disruptive_inframe_insertion",
            "frameshift_variant","initiator_codon_variant","missense_variant",
            "start_lost","stop_gained","stop_lost")
sort(setdiff(tmp,coding))
coding <- sapply(coding,function(x) grep(paste0("\\b",x,"\\b"),Final[,"snpEff_func"]))
coding <- sort(unique(unlist(coding)))
Final[coding,"effectSnpEff"] <- "coding"
splicing <- c("splice_acceptor_variant","splice_donor_variant","splice_region_variant")
splicing <- sapply(splicing,function(x) grep(paste0("\\b",x,"\\b"),Final[,"snpEff_func"]))
splicing <- sort(unique(unlist(splicing)))
Final[rownames(Final)%in%splicing & Final$effectSnpEff==".","effectSnpEff"] <- "splicing"
Final$effectSnpEff[Final$effectSnpEff=="."] <- "noncoding"
table(Final$effectSnpEff)
# coding noncoding  splicing 
# 13201      8874      5808 
#Annovar
tmp <- unique(Final[,"Func.refGene"])
tmp <- sort(tmp)
paste0(tmp,collapse="','")
tmp <- c('downstream','frameshift deletion','frameshift insertion','intergenic','intronic',
         'ncRNA_exonic','ncRNA_exonic;splicing','ncRNA_intronic','ncRNA_splicing',
         'nonframeshift deletion','nonframeshift insertion','nonsynonymous SNV','splicing',
         'stopgain','stoploss','synonymous SNV','unknown','upstream','upstream;downstream',
         'UTR3','UTR5','UTR5;UTR3')
Final[Final$Func.refGene%in%c('frameshift deletion','frameshift insertion','frameshift substitution',
                              'nonframeshift deletion','nonframeshift insertion','nonframeshift substitution',
                              'nonsynonymous SNV','stopgain','stoploss'),"effectAnnovar"] <- "coding" 
Final[Final$Func.refGene%in%c('splicing','ncRNA_exonic;splicing'),"effectAnnovar"] <- 'splicing' 
Final[Final$effectAnnovar==".","effectAnnovar"] <- "noncoding"
table(Final$effectAnnovar)
# 
# coding noncoding  splicing 
# 11149     16450       284 

Final <- data.frame(effectFilt="reject",Final,stringsAsFactors = FALSE)
Final[Final$effectSnpEff%in%c("coding","splicing")|Final$effectAnnovar%in%c("coding","splicing"),"effectFilt"] <- "keep"
table(Final$effectFilt)
# keep reject 
# 19054   8829 

# Annotation of gene paralogs with high homology (http://massgenomics.org/2013/06/ngs-false-positives.html)

In [ ]:
%%R
Final <- data.frame(blackGenes=".",Final,stringsAsFactors = FALSE)
artifacts <- c("LOC","ENS","FAM","GOL","PRA","NBP","POT","DEF","MUC","KRT","WAS","ANK","TRI","FRG","HLA",paste0("OR",1:9))
# artifacts <- c("LOC","ENS","FAM","GOL","PRA","NBP","DEF","MUC","KRT","WAS","ANK","FRG","HLA",paste0("OR",1:9)) #KEEP POT AND TRIM
genesSuff <- sapply(Final[,"Gene.refGene"],function(x) substring(unlist(strsplit(x,","))[1],1,3))
toRemove <- genesSuff%in%artifacts
Final[toRemove,"blackGenes"] <- "reject"
genesSuff <- sapply(Final[,"Gene.refGene"],function(x) substring(unlist(strsplit(x,","))[1],1,4))
toRemove <- genesSuff%in%c("PLIN","CELA","SRA1")
Final[toRemove,"blackGenes"] <- "reject"
artifacts <- c("ATXN1","PBRM1","ZNF814","MSH3","TTN","USH2A")
genes <- sapply(Final[,"Gene.refGene"],function(x) unlist(strsplit(x,","))[1])
toRemove <- genes%in%artifacts
Final[toRemove,"blackGenes"] <- "reject"
table(Final$blackGenes)
# . reject 
# 22994   4889 

# ExAC, 1000Genomes, esp6500siv2_all, Kaviar_AF, HRC_AF allele frequency < 5% annotation

In [ ]:
%%R
Final <- data.frame(SNP=".",Final,stringsAsFactors = FALSE)
toReject <- as.numeric(Final[,"ExAC_ALL"])>=0.05
toReject[is.na(toReject)] <- FALSE
table(toReject)
# FALSE  TRUE 
# 21888  5995  
Final[toReject,"SNP"] <- "reject"
toReject <- as.numeric(Final[,"X1000g2015aug_all"])>=0.05
toReject[is.na(toReject)] <- FALSE
table(toReject)
# FALSE   TRUE 
# 25356  2527 
Final[toReject,"SNP"] <- "reject"
toReject <- as.numeric(Final[,"esp6500siv2_all"])>=0.05
toReject[is.na(toReject)] <- FALSE
table(toReject)
# FALSE   TRUE 
# 26081  1802
Final[toReject,"SNP"] <- "reject"
toReject <- as.numeric(Final[,"Kaviar_AF"])>=0.05
toReject[is.na(toReject)] <- FALSE
table(toReject)
# FALSE   TRUE 
# 26529  1354 
Final[toReject,"SNP"] <- "reject"
toReject <- as.numeric(Final[,"HRC_AF"])>=0.05
toReject[is.na(toReject)] <- FALSE
table(toReject)
# FALSE   TRUE 
# 27154   729 
Final[toReject,"SNP"] <- "reject"
table(Final$SNP)
# . reject 
#  20729   7154  

# snp138NonFlagged annotation

In [ ]:
%%R
sum(toRemove <- Final[,"snp138NonFlagged"]!=".") #8935
Final[toRemove,"SNP"] <- "reject"
table(Final$SNP)
# . reject 
# 15986  11897 

# 50-mer alignability annotation

In [ ]:
%%R
Final <- data.frame(fiftymer=".",Final,stringsAsFactors = FALSE)
bed <- Final[,c("Chr","Start","End")]
write.table(bed,file='Final.bed',col.names=FALSE,row.names=FALSE,quote=FALSE,sep='\t')

# UNIX
!bedtools intersect -wo -a Final.bed -b /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/Homo_sapiens/UCSC/hg19/mappability.50mer.hg19.txt > 50merScore.bed

score <- read.delim("50merScore.bed",header=FALSE,as.is=TRUE)
sum(score[,1]=='')
# 0
# score <- score[!score[,1]=='',c(1,2,3,7)]
score <- score[,c(1,2,3,7)]
id <- paste0(score[,1],"-",score[,2])
sum(duplicated(id))
score <- score[!duplicated(id),]
id <- paste0(score[,1],"-",score[,2])
rownames(score) <- id
ID <- paste0(bed[,1],"-",bed[,2])
Score <- score[ID,4]
Final <- cbind(Score,Final)
sum(is.na(Score))
#0
# View(Final[is.na(Score),]) #strange contigs (ie: chr1_gl000192_random)
toRemove <- which(as.numeric(Final[,'Score'])<0.3)
Final[toRemove,"fiftymer"] <- "reject"
table(Final$fiftymer)
# . reject 
# 22962   4921 

# Masked region annotation

In [ ]:
%%R
bed <- Final[,c("Chr","Start","End")]
write.table(bed,file='Finalmask.bed',col.names=FALSE,row.names=FALSE,quote=FALSE,sep='\t')
# UNIX
# bedtools intersect -wao -a Finalmask.bed -b /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/Homo_sapiens/1000Genomes/20141020.pilot_mask.whole_genome.bed > Pilotmask.bed
score <- read.delim("Pilotmask.bed",header=FALSE,as.is=TRUE)
id <- paste0(score[,1],":",score[,2],"-",score[,3])
table(duplicated(id))
score <- score[!duplicated(id),]
id <- paste0(score[,1],":",score[,2],"-",score[,3])
rownames(score) <- id
ID <- paste0(bed[,1],":",bed[,2],"-",bed[,3])
table(ID%in%id)
Score <- score[ID,7]
Final <- data.frame(MaskedFilter=Score,Final,stringsAsFactors=FALSE)
Final[Final[,"MaskedFilter"]=="pilot","MaskedFilter"] <- "pass"
Final[Final[,"MaskedFilter"]==".","MaskedFilter"] <- "fail"
table(Final$MaskedFilter)
# fail   pass 
#  5656 22227 

# FLAGS genes annotation (https://bmcmedgenomics.biomedcentral.com/articles/10.1186/s12920-014-0064-y#Sec29)

In [ ]:
%%R
Final <- data.frame(FLAGS=".",FLAGSscore=0,Final,stringsAsFactors = FALSE)
FLAGS <- read.delim("/storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/FLAGS_12920_2017_309_MOESM3_ESM.txt",sep="\t",as.is=TRUE,header = FALSE)
# quantile(FLAGS[,2])
# # 0%  25%  50%  75% 100% 
# # 1   15   28   49 2659 
# quantile(FLAGS[,2],0.99) #195
# View(FLAGS[FLAGS[,2]>195,])
tmp <- FLAGS$V1
# sum(duplicated(tmp))
# tmp[duplicated(tmp)]
# # "01-Mar" "02-Mar"
# View(FLAGS[order(tmp),])
FLAGS <- FLAGS[!duplicated(tmp),]
rownames(FLAGS) <- FLAGS$V1
genes <- sapply(Final[,"Gene.refGene"],function(x) unlist(strsplit(x,","))[1])
Final$FLAGSscore <- FLAGS[genes,2]
table(is.na(Final$FLAGSscore))
Final$FLAGSscore[is.na(Final$FLAGSscore)] <- 0
# View(Final[order(Final[,1],decreasing=TRUE),])
sum(Final[,"FLAGSscore"]>195) #2606
Final[Final[,"FLAGSscore"]>195,"FLAGS"] <- "reject"
table(Final$FLAGS)
# . reject 
# 25277   2606
save(Final,file="Final.tmp.rda")

# cBio Annotation

In [ ]:
%%R
bed <- Final[,c("Chr","Start","End")]
write.table(bed,file="SomaticVariants.bed",col.names=FALSE,row.names=FALSE,quote=FALSE,sep='\t')
# UNIX
# bedtools intersect -wao -f 1 -a SomaticVariants.bed -b /storage/gluster/vol1/data/PUBLIC/NGS_ReferenceDatabase/Homo_sapiens/cBioDb/cBioMut.bed  > SomaticVariants_cBio.bed
selVar_cBio <- read.delim("SomaticVariants_cBio.bed",header=FALSE,as.is=TRUE)
load("/storage/gluster/vol1/data/BIOINFO/fulvio/cBioDb/cBioMutDb.RData",verbose=TRUE)
ID <- paste0(selVar_cBio[,1],":",selVar_cBio[,2],"-",selVar_cBio[,3])
id <- paste0(Final[,"Chr"],":",Final[,"Start"],"-",Final[,"End"])

cBio <- function(x){
  cBioID <- as.numeric(sub("cBio_","",unique(selVar_cBio[ID==x,7])))
  a <- paste0(cBioID,collapse="|")
  b <- paste0(cBioMutDb[cBioID,"AAchange"],collapse="|")
  c <- paste0(cBioMutDb[cBioID,"KnownVar"],collapse="|")
  d <- paste0(cBioMutDb[cBioID,"StudyID"],collapse="|")
  e <- sum(as.numeric(cBioMutDb[cBioID,"SamplesNum"]))
  f <- paste0(cBioMutDb[cBioID,"Freq"],collapse="|")
  e <- c(a,b,c,d,e,f)
  return(e)
}

cl <- makePSOCKcluster(20,outfile="")
registerDoParallel(cl)
#clusterExport(cl, c("cBioMutDb","id"))
cBioAnn <- foreach(i = 1:length(id)) %dopar% {
  tmp <- cBio(id[i])
  return(tmp)
}
stopCluster(cl)

cBioAnn <- do.call(rbind,cBioAnn)
nrow(cBioAnn) #27883
colnames(cBioAnn) <- c("cBioId","cBioAAchange","cBioKnownVar","cBioStudyId","cBioSamplesNum","cBioFreq")
Final <- data.frame(Final,cBioAnn,stringsAsFactors = FALSE)
# View(Final[cBioAnn[,1]!="NA",])

# Onco-annotation

In [ ]:
%%R
census <- read.delim("/storage/gluster/vol1/data/BIOINFO/fulvio/Columbia/oncoAnnotation/Census_allFri Nov  3 17_33_51 2017.tsv",as.is=TRUE)
nature <- read.delim("/storage/gluster/vol1/data/BIOINFO/fulvio/Columbia/oncoAnnotation/nature12981-s1.txt",as.is=TRUE)
oncoKB <- read.delim("/storage/gluster/vol1/data/BIOINFO/fulvio/Columbia/oncoAnnotation/oncoKB_allAnnotatedVariants.txt",as.is=TRUE)
oncoKB <- oncoKB[oncoKB$Oncogenicity=="Oncogenic",]

onco <- function(i){
  x <- Final[i,]
  gene <- unlist(strsplit(as.character(x["Gene.refGene"]),";"))[1]
  #census
  if(gene%in%census$Gene.Symbol){
    a <- census[census$Gene.Symbol%in%gene,"Role.in.Cancer"]
    b <- paste0(census[census$Gene.Symbol%in%gene,c("Tumour.Types.Germline.","Cancer.Syndrome")],collapse="|")
  } else {
    a <- "."
    b <- "."
  }
  #nature
  if(gene%in%nature$Gene..Symbol){
    c <- paste0(nature[nature$Gene..Symbol%in%gene,"Mechanism.of.action.of.CPG.mutations"],collapse="|")
    d <- nature[nature$Gene..Symbol%in%gene,"Cancer.syndrome.s."]
  } else {
    c <- "."
    d <- "."
  }
  #oncoKB
  if(gene%in%oncoKB$Gene) {
    e <- paste0(oncoKB[oncoKB$Gene%in%gene,"Mutation.Effect"],collapse="|")
    f <- paste0(oncoKB[oncoKB$Gene%in%gene,"Alteration"],collapse="|")
  } else {
    e <- "."
    f <- "."
  }
  g <- c(a,b,c,d,e,f)
  return(g)
}

cl <- makePSOCKcluster(30,outfile="")
registerDoParallel(cl)
Onco <- foreach(i = 1:nrow(Final)) %do% {
  tmp <- onco(i)
  return(tmp)
}
stopCluster(cl)

Onco <- do.call(rbind,Onco)
nrow(Onco) #27883
colnames(Onco) <- c("census","censusGermline","nature","natureGermline","oncoKB","oncoKBmut")

Final <- data.frame(Final,Onco ,stringsAsFactors = FALSE)
#save(Final,file="Final.tmp.rda")

# Panglioma recurrent mutations

In [ ]:
%%R
smg <- read.csv("/storage/gluster/vol1/data/BIOINFO/fulvio/Columbia/mmc30.csv",as.is=TRUE)
genes <- sapply(Final[,"Gene.refGene"],function(x) unlist(strsplit(x,","))[1])
smgOcc <- NULL
for(i in 1:nrow(Final)){
  if(genes[i]%in%smg[,1]){
    smgOcc <- c(smgOcc,smg[smg[,1]==genes[i],2])
  }
  else smgOcc <- c(smgOcc,0)
}
Final <- data.frame(Final,smgOcc,stringsAsFactors = FALSE)

# Damaging prediction counts

In [ ]:
%%R
# pred <- Final[,colnames(Final)[grep("_pred",colnames(Final))]]
# pred <- pred[,c("SIFT_pred","Polyphen2_HDIV_pred","MutationTaster_pred","PROVEAN_pred")]
unique(Final$SIFT_pred) # "D" "." "T"
unique(Final$Polyphen2_HDIV_pred) #"P" "." "B" "D"
Final$Polyphen2_HDIV_pred[Final$Polyphen2_HDIV_pred=="P"] <- "D"
Final$Polyphen2_HDIV_pred[Final$Polyphen2_HDIV_pred=="B"] <- "T"
unique(Final$MutationTaster_pred) #"D" "." "N" "A" "P"
Final$MutationTaster_pred[Final$MutationTaster_pred=="A"] <- "D"
Final$MutationTaster_pred[Final$MutationTaster_pred=="N"] <- "T"
Final$MutationTaster_pred[Final$MutationTaster_pred=="P"] <- "T"
unique(Final$PROVEAN_pred) #"N" "." "D"
Final$PROVEAN_pred[Final$PROVEAN_pred=="N"] <- "T"
tmp <- apply(Final[,c("SIFT_pred","Polyphen2_HDIV_pred","MutationTaster_pred","PROVEAN_pred")],1,function(x){
  d <- sum(x%in%c("D"))
  t <- sum(x%in%c("T"))
  u <- sum(x%in%c("."))
  c(d,t,u)
})
tmp <- t(tmp)
colnames(tmp) <- c("Damaging","Tolerated","Unpredicted")

Final <- data.frame(Pathogenicity=".",MissenseDamaging=tmp[,1],Final,stringsAsFactors = FALSE)

table(Final$Func.refGene)

Final[Final$Func.refGene=="nonsynonymous SNV"&as.numeric(Final$MissenseDamaging)>=2,"Pathogenicity"] <- "pathogenic missense"
Final[Final$Func.refGene=="nonsynonymous SNV"&as.numeric(Final$MissenseDamaging)<2,"Pathogenicity"] <- "tolerated missense"

Final[Final$Pathogenicity=="."&as.numeric(Final$MissenseDamaging)>=2,"Pathogenicity"] <- "pathogenic"

Final[Final$Func.refGene%in%c("frameshift deletion","frameshift insertion","frameshift substitution"),"Pathogenicity"] <- "frameshift"
Final[Final$Func.refGene%in%c("frameshift deletion","frameshift insertion","frameshift substitution"),
      c("MissenseDamaging","SIFT_pred","Polyphen2_HDIV_pred","MutationTaster_pred","PROVEAN_pred")] <- "."

Final[Final$Func.refGene%in%c("splicing","exonic;splicing"),"Pathogenicity"] <- "splicing"
Final[Final$Func.refGene%in%c("splicing","exonic;splicing"),
      c("MissenseDamaging","SIFT_pred","Polyphen2_HDIV_pred","MutationTaster_pred","PROVEAN_pred")] <- "."

Final[Final$Func.refGene%in%c("stopgain"),"Pathogenicity"] <- "truncating"
Final[Final$Func.refGene%in%c("stopgain"),
      c("MissenseDamaging","SIFT_pred","Polyphen2_HDIV_pred","MutationTaster_pred","PROVEAN_pred")] <- "."

Final[Final$Func.refGene%in%c("stoploss"),"Pathogenicity"] <- "stoploss"
Final[Final$Func.refGene%in%c("stoploss"),
      c("MissenseDamaging","SIFT_pred","Polyphen2_HDIV_pred","MutationTaster_pred","PROVEAN_pred")] <- "."

Final[Final$Func.refGene%in%c("nonframeshift deletion","nonframeshift insertion","nonframeshift substitution"),"Pathogenicity"] <- "inframe"
Final[Final$Func.refGene%in%c("nonframeshift deletion","nonframeshift insertion","nonframeshift substitution"),
      c("MissenseDamaging","SIFT_pred","Polyphen2_HDIV_pred","MutationTaster_pred","PROVEAN_pred")] <- "."

table(Final$Pathogenicity)
# .          frameshift             inframe          pathogenic pathogenic missense            splicing            stoploss  tolerated missense 
# 16427                 519                1349                  23                3549                 284                   7                5400 
# truncating 
# 325 

# DOWNSTREAM SELECTION

In [ ]:
%%R
# 1000Genomes and dbNSFP filter AF ≤ 0.0004
Final <- data.frame(PopFreq=".",Final,stringsAsFactors = FALSE)
toReject <- as.numeric(Final[,"ExAC_ALL"])>0.0004
toReject[is.na(toReject)] <- FALSE
Final[toReject,"PopFreq"] <- "reject"
toReject <- as.numeric(Final[,"X1000g2015aug_all"])>0.0004
toReject[is.na(toReject)] <- FALSE
Final[toReject,"PopFreq"] <- "reject"
table(Final$PopFreq)
# . reject 
# 15887  11996  

# Low confidence n_alt_count > 1 | t_depth < 15 | t_alt_count <= 3 | T_vaf < 0.05
Final <- data.frame(LowConf=".",Final,stringsAsFactors = FALSE)

sum(sel <- is.na(Final$n_vaf))
unique(Final[sel,"Patient_ID"])
# sum(sel <- grepl("NA",Final$n_vaf))
# unique(Final[sel,"caller"])

lowConf <- function(i){
  if(!is.na(Final$n_vaf[i])){
    tmp <- Final$n_alt_count[i]
    # tmp <- as.numeric(unlist(strsplit(Final$n_alt_count[i],"|",fixed=TRUE)))
    # if(length(tmp)>1){
    #   toRej1 <- sum(tmp>1)>1
    # } else {
    toRej1 <- as.numeric(tmp)>1
    # } 
  } else toRej1 <- FALSE
  tmp <- Final$t_depth[i]
  # tmp <- as.numeric(unlist(strsplit(Final$t_depth[i],"|",fixed=TRUE)))
  # if(length(tmp)>1) {
  #   toRej2 <- sum(tmp<15)>1
  # } else {
  toRej2 <- tmp<15
  # }
  tmp <- Final$t_alt_count[i]
  # tmp <- as.numeric(unlist(strsplit(Final$t_alt_count[i],"|",fixed=TRUE)))
  # if(length(tmp)>1) {
  #   toRej3 <- sum(tmp<=3)>1
  # } else {
  toRej3 <- tmp<=3
  # }
  tmp <- Final$t_vaf[i]
  # tmp <- as.numeric(unlist(strsplit(Final$t_vaf[i],"|",fixed=TRUE)))
  # if(length(tmp)>1) {
  #   toRej4 <- sum(tmp<0.05)>1
  # } else {
  toRej4 <- tmp<=0.05
  # }
  toRej <- toRej1|toRej2|toRej3|toRej4
  return(toRej)
}

# res <- NULL
# for(i in 1:nrow(Final)){
#   print(i)
#   tmp <- lowConf(i)
#   res <- c(res,tmp)
# }


cl <- makePSOCKcluster(30,outfile="")
registerDoParallel(cl)
res <- foreach(i = 1:nrow(Final)) %dopar% {
  tmp <- lowConf(i)
  return(tmp)
}
stopCluster(cl)
res <- unlist(res)

table(res)
# FALSE   TRUE 
# 12495 15388 
# table(is.na(res))
Final[res,"LowConf"] <- "reject"
table(Final$LowConf)
# . reject 
#  12495  15388 

# TABLE FINALIZATION

In [ ]:
%%R
# tmp <- paste0(colnames(Final),collapse = "','")
# 
# tmp <- c('Description','LowConf','PopFreq','Pathogenicity','MissenseDamaging','FLAGSscore','FLAGS',
#          'MaskedFilter','Score','fiftymer','SNP','blackGenes','effectFilt','effectSnpEff','effectAnnovar','Patient_ID',
#          'Type','Chr','Start','End','Ref','Alt','Func.refGene','Gene.refGene','GeneDetail.refGene','Interpro_domain',
#          'avsnp150','snp138NonFlagged','ExAC_ALL','X1000g2015aug_all','esp6500siv2_all','Kaviar_AF','HRC_AF','cosmic80','CLINSIG',
#          'CLNDBN','CLNACC','CLNDSDB','CLNDSDBID','SIFT_pred','Polyphen2_HDIV_pred','MutationTaster_pred','PROVEAN_pred',
#          'dbscSNV_ADA_SCORE','dbscSNV_RF_SCORE','dann','Eigen','gerp..gt2','CADD','CADD_Phred',
#          'snpEff','snpEff_func','caller','t_depth','t_ref_count','t_alt_count','n_depth','n_ref_count','n_alt_count','t_vaf','n_vaf',
#          'FPfilter','FPfail','cBio','cBioId','cBioAAchange','cBioKnownVar','cBioStudyId','cBioSamplesNum','cBioFreq',
#          'census','censusGermline','nature','natureGermline','oncoKB','oncoKBmut')

tmp <- colnames(Final)

sel <- c("Patient_ID",#'Tumor_ID',"Normal_ID", #SAMPLE
         'caller','callerN',"duplicated",'Type','Chr','Start','End','Ref','Alt',
         #"HGVSc","HGVSp", #VARIANT
         'Gene.refGene','Description','Func.refGene','GeneDetail.refGene','snpEff','snpEff_func','Interpro_domain','effectFilt','effectSnpEff','effectAnnovar', #GENE ANNOTATION
         'SNP','PopFreq','avsnp150','snp138NonFlagged','ExAC_ALL','X1000g2015aug_all','esp6500siv2_all','Kaviar_AF','HRC_AF', #POPULATION FREQUENICIES
         'FPfilter','FPfail','MaskedFilter','Score','fiftymer','FLAGSscore','FLAGS','blackGenes', #FALSE POSITIVE FILTERS
         #"VN_occurrences","VN_frequency","VN_fullycalled_count","VN_fullycalled_frequency","vn",
         "PON_filter","PON_filter_pass", #FALSE POSITIVE FILTERS
         'LowConf','t_depth','t_ref_count','t_alt_count','n_depth','n_ref_count','n_alt_count','t_vaf','n_vaf', #READ COUNTS
         'Pathogenicity','MissenseDamaging','SIFT_pred','Polyphen2_HDIV_pred','MutationTaster_pred','PROVEAN_pred', #PATHOGENICITY
         'dbscSNV_ADA_SCORE','dbscSNV_RF_SCORE','dann','Eigen','gerp..gt2','CADD','CADD_Phred', #WGS PATHOGENICITY
         'cosmic80','CLINSIG','CLNDBN','CLNACC','CLNDSDB','CLNDSDBID', #CLINICAL ANNOTATION
         'census','censusGermline','nature','natureGermline','oncoKB','oncoKBmut',"smgOcc", #ONCO ANNOTATION
         'cBioId','cBioAAchange','cBioKnownVar','cBioStudyId','cBioSamplesNum','cBioFreq' #cBio ANNOTATION
)
setdiff(tmp,sel)
setdiff(sel,tmp)
Final <- Final[,sel]

Final <- Final[order(as.numeric(Final$Start)),]
Final <- Final[order(Final$Gene.refGene),]
#Final <- Final[order(Final$Tumor_ID),]
rownames(Final) <- NULL
# tmp <- sapply(Final$Patient_ID, function(x) unlist(strsplit(x,"_"))[1])
# Final <- Final[order(tmp),]

Final[Final==""] <- "."

# HGVS from SNPEFF

In [ ]:
%%R
snpEff <- sapply(Final$snpEff,function(x){
  temp <- unlist(strsplit(x,"ANN"))[2]
  # temp <- unlist(strsplit(temp,","))[1]
  temp <- unlist(strsplit(temp,","))
  if(sum(grepl("p.",temp,fixed=TRUE))>0){
    temp <- temp[grep("p.",temp,fixed=TRUE)][1]
  } else temp <- temp[1]
  info <- unlist(strsplit(temp,"|",fixed=TRUE))
  HGVSc <- info[grep("c.",info,fixed=TRUE)][1]
  HGVSp <- info[grep("p.",info,fixed=TRUE)][1]
  return(c(HGVSc,HGVSp))
})
dim(snpEff)
snpEff <- t(snpEff)
colnames(snpEff) <- c("HGVSc","HGVSp")
rownames(snpEff) <- NULL
Final <- data.frame(snpEff,Final,stringsAsFactors = FALSE)
# Final[,c("HGVSc","HGVSp")] <- snpEff[,c("HGVSc","HGVSp")]

In [ ]:
%%R
Final <- data.frame(selected=".",Final,stringsAsFactors = FALSE)
Final$selected[grepl("path",Final$CLINSIG,ignore.case = TRUE)] <- "yes"
Final$selected[Final$Gene.refGene%in%c("NF1","SUZ12","EED")] <- "yes"
table(Final$selected) # 36 
# View(Final[Final$selected=="yes",])
reject <- Final$effectFilt=="reject" |  Final$SNP=="reject" | Final$fiftymer=="reject" | Final$FLAGS=="reject" | Final$blackGenes=="reject"
# View(Final[reject&Final$selected=="yes",])
table(reject&Final$selected=="yes") #3
Final$selected[reject&Final$selected=="."] <- "no"
table(Final$selected)
# .    no   yes 
# 8591 19261    36 

FinalS <- Final[Final$selected!="no",]
rownames(FinalS) <- NULL

table(FP_1 <- FinalS$FPfilter=="FAIL" & FinalS$MaskedFilter=="fail") #472
table(FP_tmp <- FinalS$PopFreq=="reject" | FinalS$FPfilter=="FAIL" | FinalS$MaskedFilter=="fail" | FinalS$LowConf=="reject") #5707
table(FP_2 <- FP_tmp & (FinalS$PON_filter_pass==FALSE | FinalS$callerN<3)) #5329
table(FP_3 <- FinalS$effectAnnovar=="noncoding") #2414
table(FP_4 <- FinalS$FPfilter=="no_readcounts"&FinalS$callerN<3&(nchar(FinalS$Ref)>3|nchar(FinalS$Alt)>3)) #155
table(FP_5 <- FinalS$callerN<3 & FinalS$PON_filter_pass==FALSE) #3565 
#table(FP_6 <-  (FinalS$PON_filter > -10 | (FinalS$avsnp150!="." & FinalS$cosmic80=="."))) #5442  

# tmp <- FinalS[!(FP_1 | FP_2 | FP_3 | FP_4 | FP_5) & FP_6,]
# View(tmp)

table(FinalS$toCheck <- (FP_1 | FP_2 | FP_3 | FP_4 | FP_5 ) & grepl("path",FinalS$CLINSIG,ignore.case = TRUE)) #11 
#table(FinalS$toCheck <- (FP_1 | FP_2 | FP_3 | FP_4 | FP_5 | FP_6) & grepl("path",FinalS$CLINSIG,ignore.case = TRUE)) #11 

View(FinalS[FinalS$toCheck,])
sort(unique(FinalS$CLNDBN[FinalS$toCheck]))
sort(unique(FinalS$Gene.refGene[FinalS$toCheck]))

table(toReject1 <- FinalS$toCheck & !FinalS$n_alt_count%in%c("0",".")) #1
table(toReject2 <- FinalS$toCheck  & FinalS$PopFreq=="reject") #6
table(toKeep <- FinalS$toCheck & !(toReject1 | toReject2)) #5
table(toKeep <- toKeep | (FinalS$toCheck & FinalS$Gene.refGene%in%c("TP53","IDH1"))) #5

View(FinalS[toKeep,])

FP <- (FP_1 | FP_2 | FP_3 | FP_4 | FP_5) & !toKeep
#FP <- (FP_1 | FP_2 | FP_3 | FP_4 | FP_5 | FP_6) & !toKeep

table(FP)
# FALSE  TRUE 
#  1109  6459 

FinalF <- FinalS[!FP,]
FinalS <- data.frame(FP=".",FinalS,stringsAsFactors = FALSE)
FinalS$FP[FP] <- "reject"
table(FinalS$FP)
#    . reject 
#  1109   6459 
# FinalF <- FinalS[FinalS$FP==".",]
nrow(FinalF) #1104
rownames(FinalF) <- NULL

### Hotspot like

In [ ]:
%%R
FinalF <- data.frame(Hotspot=".",FinalF,stringsAsFactors = FALSE)
mut <- paste0(FinalF$Chr,",",FinalF$Start)
Mut <- table(mut)
Mut <- sort(Mut,decreasing=TRUE)
Mut <- cbind(Mut,1)
for(i in 1:nrow(Mut)){
  print(i)
  tmp <- unique(FinalF[mut==rownames(Mut)[i],"Patient_ID"])
  # tmp <- unique(sapply(tmp,function(x) unlist(strsplit(x,"_"))[1]))
  Mut[i,2] <- length(tmp)
}

for(i in 1:nrow(FinalF)){
  FinalF[i,"Hotspot"] <- Mut[mut[i],2]
}
table(FinalF$Hotspot)

# 1  10  12  13  14  15  16  17  18  19   2  20   3   4   5   6   7   8 
# 42  10  12  13  28  45  48  51  18  38  22 700   9  12   5  12   7  32 

View(FinalF[FinalF$Hotspot!=1,])
table(FinalF$Gene.refGene[FinalF$Hotspot!=1])

# REDUNDANT MUTATIONS

In [ ]:
%%R
rownames(FinalF) <- NULL
id <- paste0(FinalF$Tumor_ID,"|",FinalF$Gene.refGene)
table(red <- duplicated(id))
# FALSE  TRUE 
#  112   992 
red <- id%in%id[red]
FinalF <- data.frame(red,FinalF,stringsAsFactors = FALSE)
View(FinalF[FinalF$red==TRUE,c("Patient_ID","caller","callerN","Chr","Start","End","Ref","Alt","Gene.refGene")])
toRemove <- FinalF$red & FinalF$callerN==1
View(FinalF[toRemove,c("Patient_ID","caller","callerN","Chr","Start","End","Ref","Alt","Gene.refGene")])
FinalF <- FinalF[!toRemove,]
rownames(FinalF) <- NULL
nrow(FinalF) #1078

# Mutation recurrence (correct to count only one mutation for patient)
genes <- sapply(FinalF[,"Gene.refGene"],function(x) unlist(strsplit(x,","))[1])
MutGenes <- table(genes)
MutGenes <- sort(MutGenes,decreasing=TRUE)
MutGenes <- cbind(MutGenes,1)
for(i in 1:nrow(MutGenes)){
  print(rownames(MutGenes)[i])
  tmp <- FinalF[genes==rownames(MutGenes)[i],"Patient_ID"]
  tmp <- unique(tmp)
  MutGenes[i,2] <- length(tmp)
}
FinalF <- data.frame(FinalF,GeneRec=1,GenePzRec=1,stringsAsFactors = FALSE)
for(i in 1:nrow(FinalF)){
  FinalF[i,"GeneRec"] <- MutGenes[genes[i],1]
  FinalF[i,"GenePzRec"] <- MutGenes[genes[i],2]
}
table(FinalF$GeneRec)
table(FinalF$GenePzRec)
#View(FinalF[FinalF$GenePzRec!=1,])

mutCounts <- sort(table(FinalF$Tumor_ID))

tmp <- colnames(FinalF)
sel <- c("Patient_ID", #"Tumor_ID", "Normal_ID",
         "caller",	"callerN",	"Type",	"Chr",	"Start",	"End", "Ref",	"Alt",	
         "Gene.refGene",	"Description",	"Func.refGene",	"GeneDetail.refGene","snpEff_func",	"HGVSc",	"HGVSp",	"Interpro_domain",	
         "Pathogenicity",	"MissenseDamaging",	"SIFT_pred",	"Polyphen2_HDIV_pred",	"MutationTaster_pred",	"PROVEAN_pred",	
         "census",	"censusGermline",	"nature",	"natureGermline",	"oncoKB",	"oncoKBmut",	"smgOcc",	
         "FPfilter",	"FPfail",	"MaskedFilter", "PON_filter", "PON_filter_pass", "Hotspot",
        # "VN_occurrences","vn",
         "avsnp150",	"ExAC_ALL",	"X1000g2015aug_all", "esp6500siv2_all",	"Kaviar_AF",	"HRC_AF",	
         "cosmic80",	"CLINSIG",	"CLNDBN",	"CLNACC",	"CLNDSDB",	"CLNDSDBID",
         "t_depth",	"t_ref_count",	"t_alt_count", "n_depth",	"n_ref_count",	"n_alt_count",	"t_vaf",	"n_vaf",
         "cBioId",	"cBioAAchange",	"cBioKnownVar",	"cBioStudyId",	"cBioSamplesNum",	"cBioFreq",
         "GeneRec",	"GenePzRec")
setdiff(sel, tmp)
setdiff(tmp, sel)  
FinalF <- FinalF[,sel]

save(Final,FinalS,FinalF,file="Final_no_PoN_filter.rda")
FinalS <- FinalS[,!colnames(FinalS)%in%c("snpEff")]
write.table(FinalS,file = "ABT414_SomaticVariants.txt",sep="\t",row.names = FALSE)
write.table(FinalF,file = "ABT414_SomaticVariants_FPfilt.txt",sep="\t",row.names = FALSE)
write.table(FinalF$Gene.refGene,file = "ABT414_SomaticVariants_FPfilt_genes.txt",sep="\t",row.names = FALSE)